# US Presidential Election 2020

In [1]:
import sys
import os

In [2]:
sys.path.append(os.path.abspath('src'))

In [3]:
from src import preprocessing, topic_modeling, deepseek, misc_utils

In [4]:
# Define the configuration file path
config_path = 'conf/config.yaml'

In [ ]:
# Run the preprocessing pipeline - Takes very long ~30mins
# print("Running preprocessing pipeline...")
# tweets_with_dups, tweets_no_dups = preprocessing.run_preprocessing_pipeline(config_path)

In [5]:
dtm_results = topic_modeling.topic_preprocess(config_path)

Loading dataset...
Processing the CounterVectorizer and TfidfVectorizer
Processing Biden dataset...
Processing Trump dataset...
Processing combine dataset with duplicates...
Topic Modeling Preprocessing Complete!


In [7]:
# @Anthony: Ways to get all the keys out!!!
# Just need to sum all the values of each dtm to get the word count to create word cloud
print(dtm_results.keys())
print(dtm_results['biden'].keys())
print(dtm_results['trump'].keys())
print(dtm_results['combine'].keys())

dict_keys(['vectorizer', 'biden', 'trump', 'combine'])
dict_keys(['count_dtm', 'tfidf_dtm'])
dict_keys(['count_dtm', 'tfidf_dtm'])
dict_keys(['count_dtm', 'tfidf_dtm'])
